<a href="https://colab.research.google.com/github/plumwiserim/data-analysis/blob/main/part1(20200713).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# '학군'이 좋으면, '부동산' 가격이 높을까?

## 가설

학군이 좋으면, 부동산 매매 가격이 상대적으로 높을 것이다.

## 정의(def)

* 학군이란?
    * 중학교에서 특수목적고로 진학하는 비율
* 부당산 매매 가격?
    * 아파트 거래가격

# 1. 데이터 가져오기

In [ ]:
import pandas as pd
graduate_path = f'data/part1_2018년_2차_졸업생의_진로_현황(전체).xlsx'
raw_graduate = pd.read_excel(graduate_path, sheet_name='2018_졸업생의 진로 현황(중)')

# 2. 전처리

## 2-1. 원본 데이터에서 필요한 데이터만 추출

In [ ]:
select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
graduate_data = select_col.drop(0)

## 2-2. 수치화

In [ ]:
graduate_data['졸업자'] = pd.to_numeric(graduate_data['졸업자'])
graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
graduate_data['외고'] = pd.to_numeric(graduate_data['외고'])
graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
graduate_data

,지역,학교명,학교코드,졸업자,과고,외고,총합
1,서울특별시 성북구,서울대학교사범대학부설중학교,S000003514,234.0,4.0,5.0,9.0
2,서울특별시 종로구,서울대학교사범대학부설여자중학교,S000003562,172.0,0.0,8.0,8.0
3,서울특별시 강남구,개원중학교,S010000698,214.0,2.0,9.0,11.0
4,서울특별시 서초구,경원중학교,S010000700,310.0,5.0,4.0,9.0
5,서울특별시 강남구,구룡중학교,S010000701,256.0,3.0,4.0,7.0
...,...,...,...,...,...,...,...
3454,제주특별자치도 제주시,탐라중학교,S170000844,298.0,1.0,1.0,2.0
3455,제주특별자치도 제주시,제주영지학교,S170000512,12.0,0.0,0.0,0.0
3456,제주특별자치도 제주시,제주영송학교,S170000513,21.0,0.0,0.0,0.0
3457,제주특별자치도 서귀포시,서귀포온성학교,S170000790,9.0,0.0,0.0,0.0


## 2-3. 데이터 전처리

```
Output: 지역을 '시도', '구군'으로 분리
Input : {"서울특별시 OOO" or "경상북도 OOO" Or "경상남도 OOO" }
Process:
    1) 시와 구를 '공백'을 기준으로 나눈다(split).
    2) 글자수가 4자가 아니면 앞의 두글자([:2])를 출력하고, 글자수가 4자이면 첫번째([0])글자와 세번째([2]) 글자를 합쳐서 출력
```

In [ ]:
def get_province(x):
    temp = x.split(' ')
    sido = temp[0]
    if len(sido) == 4:        
        return (sido[0] + sido[2])
    else:
        return (sido[0:2])    
    return 

graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_province)
graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])

In [ ]:
# 신정중학교
graduate_data.at[588,'시도'] = '부산' 
graduate_data.at[588,'구군'] = '기장군'
# 은풍중학교
graduate_data.at[3011,'시도'] = '경북'
graduate_data.at[3011,'구군'] = '예천군'

In [ ]:
graduate_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3458 entries, 1 to 3458
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      3456 non-null   object 
 1   학교명     3458 non-null   object 
 2   학교코드    3458 non-null   object 
 3   졸업자     3424 non-null   float64
 4   과고      3424 non-null   float64
 5   외고      3424 non-null   float64
 6   총합      3424 non-null   float64
 7   시도      3458 non-null   object 
 8   구군      3458 non-null   object 
dtypes: float64(4), object(5)
memory usage: 430.2+ KB


In [ ]:
def graduate_preprocrssing(path):
    raw_graduate = pd.read_excel(path, sheet_name='2018_졸업생의 진로 현황(중)')
    select_col = raw_graduate[['지역', '학교명','정보공시 \n 학교코드', '졸업자.2', '(특수목적고)과학고 진학자.2','(특수목적고)외고ㆍ국제고 진학자.2']]
    select_col.columns = ['지역', '학교명', '학교코드', '졸업자', '과고', '외고']
    graduate_data = select_col.drop(0)
    graduate_data['과고'] = pd.to_numeric(graduate_data['과고'])
    graduate_data['외고'] =  pd.to_numeric(graduate_data['외고']) 
    graduate_data['졸업자'] =  pd.to_numeric(graduate_data['졸업자']) 
    graduate_data['총합'] = graduate_data['과고'] + graduate_data['외고']
    
    def get_province(x):
        temp = x.split(' ')[0]
        if len(temp) != 4:
            return temp[:2]
        else:
            return temp[0] + temp[2]
    
    graduate_data['시도'] = graduate_data['지역'].dropna().apply(get_province)
    graduate_data['구군'] = graduate_data['지역'].dropna().apply(lambda x: x.split(' ')[1])
    
    graduate_data.at[588,'시도'] = '부산'
    graduate_data.at[588,'구군'] = '기장군'
    graduate_data.at[3011,'시도'] = '경북'
    graduate_data.at[3011,'구군'] = '예천군'
    
    return graduate_data

In [ ]:
gradu_df = graduate_preprocrssing(graduate_path)

In [ ]:
gradu_df

,지역,학교명,학교코드,졸업자,과고,외고,총합,시도,구군
1,서울특별시 성북구,서울대학교사범대학부설중학교,S000003514,234.0,4.0,5.0,9.0,서울,성북구
2,서울특별시 종로구,서울대학교사범대학부설여자중학교,S000003562,172.0,0.0,8.0,8.0,서울,종로구
3,서울특별시 강남구,개원중학교,S010000698,214.0,2.0,9.0,11.0,서울,강남구
4,서울특별시 서초구,경원중학교,S010000700,310.0,5.0,4.0,9.0,서울,서초구
5,서울특별시 강남구,구룡중학교,S010000701,256.0,3.0,4.0,7.0,서울,강남구
...,...,...,...,...,...,...,...,...,...
3454,제주특별자치도 제주시,탐라중학교,S170000844,298.0,1.0,1.0,2.0,제주,제주시
3455,제주특별자치도 제주시,제주영지학교,S170000512,12.0,0.0,0.0,0.0,제주,제주시
3456,제주특별자치도 제주시,제주영송학교,S170000513,21.0,0.0,0.0,0.0,제주,제주시
3457,제주특별자치도 서귀포시,서귀포온성학교,S170000790,9.0,0.0,0.0,0.0,제주,서귀포시


In [ ]:
gradu_sido = gradu_df.groupby('시도').sum()
gradu_sido.sort_values(by='총합', ascending=False)

,졸업자,과고,외고,총합
시도,,,,
경기,120572.0,361.0,2183.0,2544.0
서울,76687.0,535.0,1449.0,1984.0
부산,27044.0,230.0,767.0,997.0
인천,26046.0,181.0,559.0,740.0
경남,31481.0,195.0,342.0,537.0
대전,14988.0,100.0,261.0,361.0
충북,14831.0,51.0,203.0,254.0
대구,22842.0,98.0,153.0,251.0
울산,10759.0,76.0,165.0,241.0


In [ ]:
gradu_sido['진학률'] = gradu_sido['총합'] / gradu_sido['졸업자'] * 100
gradu_sido.sort_values(by='진학률', ascending=False)

,졸업자,과고,외고,총합,진학률
시도,,,,,
부산,27044.0,230.0,767.0,997.0,3.686585
세종,2702.0,18.0,69.0,87.0,3.219837
인천,26046.0,181.0,559.0,740.0,2.841127
서울,76687.0,535.0,1449.0,1984.0,2.587140
대전,14988.0,100.0,261.0,361.0,2.408594
울산,10759.0,76.0,165.0,241.0,2.239985
제주,6487.0,36.0,102.0,138.0,2.127332
경기,120572.0,361.0,2183.0,2544.0,2.109943
충북,14831.0,51.0,203.0,254.0,1.712629
